<a href="https://colab.research.google.com/github/yyduyuxuan/Machine-Learning-for-Data-Driven-Inventory-Replenishment-Evidence-from-the-M5-Retail-Dataset/blob/main/ML_(R%2CS)_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data


In [ ]:
import pandas as pd
import os
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/Supervised Project/sales_long_val_5y.pkl'
sales_long_val = pd.read_pickle(file_path)

print(sales_long_val.shape)
sales_long_val.head()

Mounted at /content/drive
(45974682, 119)


/usr/local/lib/python3.12/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.12/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,enc_std_store_id_dept_id,enc_mean_item_id,enc_std_item_id,enc_mean_item_id_state_id,enc_std_item_id_state_id,enc_mean_item_id_store_id,enc_std_item_id_store_id,ROP_L3,ROP_L7,ROP_L14
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3,2011-01-29,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,NaN,NaN,NaN
1,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,0,2011-01-30,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,NaN,NaN,NaN
2,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,2011-01-31,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,5.0,NaN,NaN
3,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,4,1,2011-02-01,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,7.0,NaN,NaN
4,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,5,4,2011-02-02,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,6.0,NaN,NaN


In [ ]:
# Get test set observation
import pandas as pd

df = sales_long_val.copy()
df['date'] = pd.to_datetime(df['date'])

df['unique_id'] = df['item_id'].astype(str) + '_' + df['store_id'].astype(str)

START = pd.Timestamp('2015-01-29')
END   = pd.Timestamp('2016-01-28')

demand_1y = (
    df.loc[(df['date'] >= START) & (df['date'] <= END),
           ['unique_id', 'date', 'sales', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id', 'd']]
      .sort_values(['unique_id', 'date'])
      .reset_index(drop=True)
)

print(demand_1y.shape)
display(demand_1y.head())

(11028855, 9)


,unique_id,date,sales,item_id,store_id,dept_id,cat_id,state_id,d
0,FOODS_1_001_CA_1,2015-01-29,1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,1462
1,FOODS_1_001_CA_1,2015-01-30,0,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,1463
2,FOODS_1_001_CA_1,2015-01-31,3,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,1464
3,FOODS_1_001_CA_1,2015-02-01,0,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,1465
4,FOODS_1_001_CA_1,2015-02-02,0,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,1466


In [ ]:
S_ml = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/S_ml_1y.csv")
S_ml.head()

,Unnamed: 0,unique_id,model,ds,item_id,store_id,dept_id,y_true,mu_hat_H,sigma_day,SS,S
0,0,FOODS_1_145_CA_1,ML_ens,2015-01-31,FOODS_1_145,CA_1,FOODS_1,4.0,4.880773,2.949851,4.852505,10
1,1,FOODS_1_145_CA_1,ML_ens,2015-02-01,FOODS_1_145,CA_1,FOODS_1,3.0,4.401682,2.949851,4.852505,10
2,2,FOODS_1_145_CA_1,ML_ens,2015-02-02,FOODS_1_145,CA_1,FOODS_1,2.0,5.056127,2.949851,4.852505,10
3,3,FOODS_1_145_CA_1,ML_ens,2015-02-03,FOODS_1_145,CA_1,FOODS_1,3.0,4.572454,2.949851,4.852505,10
4,4,FOODS_1_145_CA_1,ML_ens,2015-02-04,FOODS_1_145,CA_1,FOODS_1,2.0,5.310244,2.949851,4.852505,11


In [ ]:
uids_159 = S_ml['unique_id'].unique().tolist()
demand_1y = demand_1y[demand_1y['unique_id'].isin(uids_159)].reset_index(drop=True)
print("159 Portfolio Coverage Days Distribution：")
print(demand_1y.groupby('unique_id').size().describe())

159 Portfolio Coverage Days Distribution：
count    159.0
mean     365.0
std        0.0
min      365.0
25%      365.0
50%      365.0
75%      365.0
max      365.0
dtype: float64


In [ ]:
demand_1y.tail()

,unique_id,date,sales,item_id,store_id,dept_id,cat_id,state_id,d
58030,HOUSEHOLD_2_371_WI_3,2016-01-24,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1822
58031,HOUSEHOLD_2_371_WI_3,2016-01-25,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1823
58032,HOUSEHOLD_2_371_WI_3,2016-01-26,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1824
58033,HOUSEHOLD_2_371_WI_3,2016-01-27,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1825
58034,HOUSEHOLD_2_371_WI_3,2016-01-28,2,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1826


In [ ]:
demand_1y.to_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/demand_1y.csv")

# (R,S) Simulation

In [ ]:
import numpy as np
import pandas as pd

# Simulation function
def simulate_RS_basestock(
    demand_series: pd.Series,
    target_series: pd.Series,
    init_onhand: float,
    lead_time: int,
    review_period: int = 1,
    label: str = "",
    model_name: str | None = None,
):
    if (not label) and model_name:
        label = str(model_name)

    if len(demand_series) != len(target_series):
        raise ValueError("Demand and target length mismatch.")
    if lead_time < 0 or review_period <= 0:
        raise ValueError("lead_time>=0 and review_period>=1 required.")

    d = demand_series.reset_index(drop=True).astype(float).to_numpy()
    s = target_series.reset_index(drop=True).astype(float).to_numpy()
    if np.isnan(d).any() or np.isnan(s).any():
        raise ValueError("Series contain NaNs.")

    N = len(d)
    start_inv  = np.zeros(N, dtype=float)
    receipt    = np.zeros(N, dtype=float)
    end_inv    = np.zeros(N, dtype=float)
    order_qty  = np.zeros(N, dtype=float)

    start_inv[0] = float(init_onhand)

    for t in range(N):
        if t > 0:
            start_inv[t] = end_inv[t-1]
        end_inv[t] = start_inv[t] + receipt[t] - d[t]

        # in trainst inventory
        in_transit = receipt[t+1 : min(t+lead_time, N)].sum() if lead_time >= 2 else 0.0

        # Periodic inspection：t = 0, R, 2R, ...
        if (t % review_period) == 0:
            ip = end_inv[t] + in_transit  # inventory position
            if ip < s[t]:
                qty = s[t] - ip
                order_qty[t] = qty
                if t + lead_time < N:
                    receipt[t + lead_time] += qty

    # KPI
    backlog_mask  = end_inv < 0
    backlog_days  = int(backlog_mask.sum())
    backlog_units = float(end_inv[backlog_mask].sum())
    total_demand  = float(d.sum())
    total_orders  = float(order_qty.sum())
    order_count   = int((order_qty > 0).sum())

    pos_onhand_sum = float(end_inv[end_inv > 0].sum())
    avg_onhand     = pos_onhand_sum / N if N else 0.0
    avg_demand     = float(d.mean()) if N else 0.0
    turnover_daily = (avg_demand / avg_onhand) if avg_onhand > 1e-12 else 0.0

    fill_rate = (total_demand + backlog_units) / total_demand if total_demand > 1e-12 else 1.0
    fill_rate = max(0.0, min(1.0, fill_rate))
    psl = 1.0 - (backlog_days / N) if N else 1.0

    # CSL
    csl_ok = csl_total = 0
    if lead_time > 0:
        for t in range(N):
            if order_qty[t] > 0 and (t + lead_time - 1) < N:
                if not (end_inv[t : t + lead_time] < 0).any():
                    csl_ok += 1
                csl_total += 1
    csl = (csl_ok / csl_total) if csl_total > 0 else float("nan")

    return {
        "fill_rate":       fill_rate,
        "psl":             psl,
        "csl":             csl,
        "turnover_daily":  turnover_daily,
        "order_count":     order_count,
        "avg_order_qty":   (total_orders / order_count) if order_count else 0.0,
    }


In [ ]:
# Batch run: Adapt to S table & demand table of 159 items
def run_rs_for_all(
    S_df: pd.DataFrame,
    D_df: pd.DataFrame,
    lead_time: int = 2,
    review_period: int = 1,
    init_policy: str | float = 'S0',
    s_col: str = 'S',
):
    S = S_df.copy()
    D = D_df.copy()

    # S：unique_id / model / ds / S
    need_S = {'unique_id', 'model', 'ds', s_col}
    miss_S = need_S - set(S.columns)
    if miss_S:
        raise ValueError(f"S_df missing: {miss_S}")

    # Demand：unique_id / ds(or date) / sales
    if 'ds' not in D.columns and 'date' in D.columns:
        D = D.rename(columns={'date': 'ds'})
    if 'demand' not in D.columns and 'sales' in D.columns:
        D = D.rename(columns={'sales': 'demand'})

    need_D = {'unique_id', 'ds', 'demand'}
    miss_D = need_D - set(D.columns)
    if miss_D:
        raise ValueError(f"D_df missing: {miss_D}")

    if not np.issubdtype(S['ds'].dtype, np.datetime64):
        S['ds'] = pd.to_datetime(S['ds'])
    if not np.issubdtype(D['ds'].dtype, np.datetime64):
        D['ds'] = pd.to_datetime(D['ds'])

    # Run on (unique_id, model)
    kpi_rows = []
    for (uid, mdl), gS in S.groupby(['unique_id', 'model'], sort=False):
        gS = gS.sort_values('ds')[['ds', s_col]].rename(columns={s_col: 'S_target'})

        gD = (D[D['unique_id'] == uid][['ds', 'demand']]
                .merge(gS[['ds']], on='ds', how='right')
                .fillna({'demand': 0.0})
                .sort_values('ds'))

        S_series = gS['S_target'].astype(float).reset_index(drop=True)
        D_series = gD['demand'].astype(float).reset_index(drop=True)

        init_onhand = float(S_series.iloc[0]) if init_policy == 'S0' else float(init_policy)

        res = simulate_RS_basestock(
            demand_series = D_series,
            target_series = S_series,
            init_onhand   = init_onhand,
            lead_time     = lead_time,
            review_period = review_period,
            model_name    = f"{uid} | {mdl}",
        )

        kpi_rows.append({
            'unique_id'      : uid,
            'model'          : mdl,
            'n_days'         : int(len(D_series)),
            'orders'         : res['order_count'],
            'avg_order_qty'  : res['avg_order_qty'],
            'fill_rate'      : res['fill_rate'],
            'PSL'            : res['psl'],
            'CSL'            : res['csl'],
            'turnover_daily' : res['turnover_daily'],
        })

    kpi = pd.DataFrame(kpi_rows).sort_values(['unique_id','model']).reset_index(drop=True)
    return kpi

In [ ]:
kpi_ml_159 = run_rs_for_all(
    S_df          = S_ml,
    D_df          = demand_1y,
    lead_time     = 2,
    review_period = 1,
    init_policy   = 'S0',            # Take S as the initial inventory on the first day
    s_col         = 'S'
)

display(kpi_ml_159.head())
print("TOTAL rows:", kpi_ml_159.shape[0])

,unique_id,model,n_days,orders,avg_order_qty,fill_rate,PSL,CSL,turnover_daily
0,FOODS_1_145_CA_1,ML_ens,359,254,2.606299,0.987897,0.991643,0.980237,0.239060
1,FOODS_1_145_CA_3,ML_ens,359,230,2.769565,0.984326,0.986072,0.956332,0.216491
2,FOODS_1_145_CA_4,ML_ens,359,195,1.851282,0.986111,0.988858,0.969231,0.225989
3,FOODS_1_145_TX_1,ML_ens,359,199,2.020101,0.997512,0.997214,0.989899,0.207967
4,FOODS_1_145_TX_2,ML_ens,359,201,2.154229,0.986207,0.991643,0.970149,0.185422


TOTAL rows: 159


In [ ]:
kpi_ml_159.to_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/kpi_ml_159.csv")